# Kuramoto XY Dynamics on Quantum Hardware

This notebook demonstrates the core mapping: **Kuramoto coupled oscillators → XY Hamiltonian → Trotterized quantum evolution**.

All cells run on the local AerSimulator — no IBM credentials needed.

## The mapping

Classical Kuramoto:
$$\frac{d\theta_i}{dt} = \omega_i + \sum_j K_{ij} \sin(\theta_j - \theta_i)$$

Quantum XY Hamiltonian:
$$H = -\sum_{i<j} K_{ij}(X_i X_j + Y_i Y_j) - \sum_i \omega_i Z_i$$

Order parameter from qubit expectations:
$$R = \frac{1}{N} \left|\sum_i (\langle X_i \rangle + i \langle Y_i \rangle)\right|$$

In [1]:
import numpy as np
from scpn_quantum_control.bridge import OMEGA_N_16, build_knm_paper27
from scpn_quantum_control.phase import QuantumKuramotoSolver

## 1. SCPN Parameters

16 natural frequencies and the coupling matrix from Paper 27.

In [2]:
n = 4
K = build_knm_paper27(L=n)
omega = OMEGA_N_16[:n]

print(f"Oscillators: {n}")
print(f"Frequencies: {omega}")
print(f"Coupling matrix:")
print(np.array2string(K, precision=3, suppress_small=True))

Oscillators: 4
Frequencies: [1.329 2.61  0.844 1.52 ]
Coupling matrix:
[[0.45  0.302 0.247 0.183]
 [0.302 0.45  0.201 0.247]
 [0.247 0.201 0.45  0.252]
 [0.183 0.247 0.252 0.45 ]]


## 2. Quantum Kuramoto Evolution

Build the XY Hamiltonian and evolve via Lie-Trotter decomposition.

In [3]:
solver = QuantumKuramotoSolver(n, K, omega)

H = solver.build_hamiltonian()
print(f"Hamiltonian: {H.num_qubits} qubits, {len(H)} Pauli terms")
print(f"First 5 terms:")
for label, coeff in zip(H.paulis[:5].to_labels(), H.coeffs[:5]):
    print(f"  {label}: {coeff.real:+.4f}")

Hamiltonian: 4 qubits, 16 Pauli terms
First 5 terms:
  IIIZ: -1.3290
  IIZI: -2.6100
  IZII: -0.8440
  ZIII: -1.5200
  IIXX: -0.3020


In [4]:
result = solver.run(t_max=1.0, dt=0.2, trotter_per_step=2)

print("Time | R (order parameter)")
print("-" * 35)
for t, R in zip(result["times"], result["R"]):
    bar = "#" * int(R * 30)
    print(f"{t:5.2f} | {R:.4f} {bar}")

C:\Users\forti\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
C:\Users\forti\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


Time | R (order parameter)
-----------------------------------
 0.00 | 0.8060 ########################
 0.20 | 0.7640 ######################
 0.40 | 0.6495 ###################
 0.60 | 0.4935 ##############
 0.80 | 0.3441 ##########
 1.00 | 0.2627 #######


## 3. Classical Reference

Compare against classical ODE integration (RK45).

In [5]:
from scpn_quantum_control.hardware.classical import classical_kuramoto_reference

cl = classical_kuramoto_reference(n, t_max=1.0, dt=0.2, K=K, omega=omega)

print(f"{'Time':>5} | {'Quantum R':>10} | {'Classical R':>12} | {'Diff':>8}")
print("-" * 45)
for t, Rq, Rc in zip(result["times"], result["R"], cl["R"]):
    print(f"{t:5.2f} | {Rq:10.4f} | {Rc:12.4f} | {abs(Rq - Rc):8.4f}")

 Time |  Quantum R |  Classical R |     Diff
---------------------------------------------
 0.00 |     0.8060 |       0.8063 |   0.0003
 0.20 |     0.7640 |       0.7825 |   0.0185
 0.40 |     0.6495 |       0.7586 |   0.1092
 0.60 |     0.4935 |       0.7343 |   0.2407
 0.80 |     0.3441 |       0.7093 |   0.3652
 1.00 |     0.2627 |       0.6834 |   0.4208


## 4. Scaling to 8 Oscillators

The same pipeline works for larger systems. Circuit depth grows linearly with qubit count.

In [6]:
n8 = 8
K8 = build_knm_paper27(L=n8)
omega8 = OMEGA_N_16[:n8]

solver8 = QuantumKuramotoSolver(n8, K8, omega8)
result8 = solver8.run(t_max=0.5, dt=0.1, trotter_per_step=2)

print(f"8-oscillator R(t): {[f'{r:.4f}' for r in result8['R']]}")

8-oscillator R(t): ['0.5918', '0.5854', '0.5662', '0.5346', '0.4915', '0.4395']


## Key takeaway

The Kuramoto→XY mapping is exact at the Hamiltonian level. Trotter decomposition introduces $O(\Delta t^2)$ error per step, and hardware decoherence adds noise proportional to circuit depth. On IBM Heron r2 (Feb 2026), the 4-qubit version achieves 7.3% error at depth 85.